In [1]:
import numpy as np
import pandas as pd

In [107]:
data = pd.DataFrame(np.array([[3,69,1],[0,69,1],[1,67,0],[3,69,0],[2,66,2],[3,67,2],[2,67,3],[0,66,3],[1,67,4],[0,72,4]]))
data.columns = ['Player_#','Score','Tournament']

In [108]:
data.sort_values('Tournament')

,Player_#,Score,Tournament
2,1,67,0
3,3,69,0
0,3,69,1
1,0,69,1
4,2,66,2
5,3,67,2
6,2,67,3
7,0,66,3
8,1,67,4
9,0,72,4


In [60]:
n_players = len(pd.unique(data['Player_#']))
n_tournaments = len(pd.unique(data.Tournament))

In [61]:
def alpha(A,a):
    wq,vq=np.linalg.eig(A)
    return a/np.amax(np.absolute(wq))

In [124]:
def run(BETA):
    a = .8
    A = np.zeros((n_players*n_tournaments,n_players*n_tournaments))
    A_alt = np.zeros((n_players*n_tournaments,n_players*n_tournaments))
    G = np.zeros((n_players*n_tournaments,n_players*n_tournaments))
    for tournament in pd.unique(data.Tournament):
        subset = data[data.Tournament==tournament].sort_values('Score')
        arrs = np.zeros((n_players,n_players))
        arrs[np.ix_(pd.unique(subset['Player_#']),pd.unique(subset['Player_#']))] = np.outer(subset.Score.values,1/subset.Score.values.astype(float))
        np.fill_diagonal(arrs,0)
        print tournament
        print np.vstack([BETA**(abs(tournament-tournament2))*arrs for tournament2 in range(len(pd.unique(data.Tournament)))])
        A_alt[:,range(n_players*tournament,n_players*(tournament+1))] += \
        np.vstack([BETA**(abs(tournament-tournament2))*arrs for tournament2 in range(len(pd.unique(data.Tournament)))])
        for u,player in enumerate(pd.unique(subset['Player_#'])):
            arr = np.zeros(n_players)
            arr1 = np.zeros(n_players)
            arr[pd.unique(subset['Player_#'])] = subset.Score.values / subset[subset['Player_#']==player].Score.values.astype(float)
            arr[pd.unique(subset['Player_#'])[u]] = 0
            arr1[pd.unique(subset['Player_#'])] = 1
            arr1[pd.unique(subset['Player_#'])[u]] = 0
            
            A[np.ix_([player + n_players*tournament2 for tournament2 in pd.unique(data.Tournament)],
                     range(n_players*tournament,n_players*(tournament+1)))] += \
            np.array([BETA**(abs(tournament-tournament2))*arr for tournament2 in pd.unique(data.Tournament)])
            
            G[np.ix_([player + n_players*tournament2 for tournament2 in pd.unique(data.Tournament)],
                     range(n_players*tournament,n_players*(tournament+1)))] += \
            np.array([BETA**(abs(tournament-tournament2))*arr1 for tournament2 in pd.unique(data.Tournament)])
    w_a = (np.linalg.inv(np.eye(n_players*n_tournaments) - alpha(A,a)*A) * np.matrix(A).sum(1)).sum(1)
    w_g = (np.linalg.inv(np.eye(n_players*n_tournaments) - alpha(G,a)*G) * np.matrix(G).sum(1)).sum(1)
    print A_alt[0]
    print A[0]
    assert np.all(A_alt==A)
    return w_a/w_g

In [125]:
run(.9)

1
[[ 0.     0.     0.     0.9  ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.9    0.     0.     0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.     0.     0.     0.9  ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.9    0.     0.     0.   ]
 [ 0.     0.     0.     0.81 ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.81   0.     0.     0.   ]
 [ 0.     0.     0.     0.729]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.729  0.     0.     0.   ]]
0
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.97101449]
 [ 0.          0.          0.          0.        ]
 [ 0.          1.02985075  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.87391304]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.9

AssertionError: 

In [78]:
arrs = np.zeros((n_players,n_players))
arrs[np.ix_([pd.unique(subset['Player_#']),pd.unique(subset['Player_#'])])] = np.outer(subset.Score.values,1/subset[subset['Player_#']==player].Score.values.astype(float))
np.fill_diagonal(arrs,0)
A[np.ix_([player + n_players*tournament2 for tournament2 in pd.unique(data.Tournament) for player in pd.unique(subset['Player_#'])],
         range(n_players*tournament,n_players*(tournament+1)))] += \
np.array([BETA**(abs(tournament-tournament2))*arrs for tournament2 in pd.unique(data.Tournament)])

NameError: name 'subset' is not defined

In [79]:
import matplotlib.pyplot as plt
%matplotlib inline

for beta in np.linspace(.5,1,5):
    rat = run(beta)
    plt.plot(range(n_tournaments),[(rat).tolist()[u][0]/(rat).tolist()[u+1][0] for u in range(0,n_tournaments*n_players,n_players)],label='Beta=%g'%beta)
plt.legend();
plt.savefig('different_betas.png')

ValueError: Cross index must be 1 dimensional